In [372]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
 
import pandas as pd
import numpy as np

from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam

# %matplotlib notebook

import matplotlib.pyplot as plt


In [373]:
df = pd.read_csv("data.csv")
data_array = df.values

data = torch.tensor(data_array, dtype=torch.float32)

In [374]:
df = pd.DataFrame(df.iloc[1:].values - df.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

In [375]:
def fuck_stack(df: pd.DataFrame, fh: int = 1):
    stacks = [df.iloc[:-fh]] + [df.iloc[i:].values if (fh == i) else df.iloc[i:-(fh - i)].values  for i in range(1, fh + 1)]
    columns = ["Time", "X", "Y", "Z"]
    for i in range(1, fh + 1):
        columns.extend([f"Time {i}", f"X {i}", f"Y {i}", f"Z {i}"])
    
    return pd.DataFrame(np.hstack(stacks), columns=columns)

In [376]:
data = torch.tensor(fuck_stack(df, 1).values, dtype=torch.float32)

In [377]:
class SimpleDataset(Dataset):

    def __init__(self, data, transform=None):
        self.data_ = data[:, :-4]

        # targets = self.data_[1:]
        # targets = torch.vstack((targets, targets[-1]))
        self.targets_ = data[:, -4:]

        # self.transform = transform

    def __len__(self):

        return len(self.data_)

    def __getitem__(self, idx):
        sample = self.data_[idx]
        target = self.targets_[idx]

        # if self.transform:
        #     sample = self.transform(sample)

        return sample, target
    def plotData(self, i = 0, f = -1):
        fig = px.line_3d(x=self.data_[i:f,1], y=self.data_[i:f,2], z=self.data_[i:f,3])
        # fig = px.scatter_3d(data, x="X", y="Y", z="Z")
        fig.show()

In [378]:
dataset = SimpleDataset(data[0:2500])
test = SimpleDataset(data[2500:3000])
dataset.plotData()

In [379]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [380]:
class MLP_1dot(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.layer_1 = nn.Linear(input, 32)
        self.act1 = nn.ReLU()

        self.layer_2 = nn.Linear(32, input)
        self.act2 = nn.ReLU()

        self.layer_3 = nn.Linear(input, output)
    
    def forward(self, X: torch.Tensor):
        X_0 = X.clone()
        X = self.layer_1(X)
        X = self.act1(X)
    
        X = self.layer_2(X)        
        X = self.act2(X + X_0)

        out = self.layer_3(X)
        return out

In [381]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 4)

    def forward(self, x):
        out, _ = self.lstm(x)
        # print(out.shape)
        out = self.linear(out)
        return out

In [382]:
# model = MLP_1dot(120,4)
model = LSTMModel(input_size=4, hidden_size=128, num_layers=1, dropout=0.0)
loss_model = nn.MSELoss(reduction='mean')
opt = Adam(model.parameters(), lr=0.001)

In [383]:
EPOCH = 100
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    train_loop = tqdm(train_loader,leave=False)
    train_loss = []
    for X, y in train_loader:
        pred = model(X)
        loss = loss_model(pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)
    
    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

  0%|          | 0/63 [00:00<?, ?it/s]

Epoch [1/100], train_loss = 0.0384, val_loss = 0.0080


Epoch [2/100], train_loss = 0.0065, val_loss = 0.0056


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch [3/100], train_loss = 0.0048, val_loss = 0.0042


...

Epoch [97/100], train_loss = 0.0012, val_loss = 0.0012


Epoch [98/100], train_loss = 0.0012, val_loss = 0.0012


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch [99/100], train_loss = 0.0012, val_loss = 0.0013


Epoch [100/100], train_loss = 0.0012, val_loss = 0.0012


In [384]:
# plt.plot(np.log(train_losses))
# plt.plot(np.log(val_losses))
# plt.legend(['train','val'])
# plt.show()

In [385]:
start, _  = dataset[0:9]
start = start[None, :]
predictions = []
model.eval()

for i in range(1,1000):
    with torch.no_grad():
        pred = model(start)
        predictions.append(pred)
        # start = torch.hstack([start[4:], pred]) 
        start = pred   

In [386]:
predictions

[tensor([[ 1.0013e+00, -1.2120e-02,  2.7586e-02, -3.2041e-04]]),
 tensor([[ 1.0012, -0.0050, -0.0054, -0.0035]]),
 tensor([[ 1.0011, -0.0078,  0.0055, -0.0030]]),
 tensor([[ 1.0011, -0.0067,  0.0019, -0.0029]]),
 tensor([[ 1.0011, -0.0071,  0.0031, -0.0031]]),
 tensor([[ 1.0011, -0.0069,  0.0027, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 tensor([[ 1.0011, -0.0070,  0.0028, -0.0030]]),
 ten

In [387]:
p_x = pd.DataFrame(map(lambda t: t.flatten().detach().tolist(), predictions), columns=["Time", "X", "Y", "Z"])
_, y  = dataset[:500]
p_y = pd.DataFrame(y.detach().numpy(), columns=["Time", "X", "Y", "Z"])

p_x = p_x.cumsum()
p_y = p_y.cumsum()

In [388]:
p_x

,Time,X,Y,Z
0,1.001255,-0.012120,0.027586,-0.000320
1,2.002418,-0.017085,0.022166,-0.003823
2,3.003532,-0.024843,0.027711,-0.006843
3,4.004621,-0.031524,0.029619,-0.009783
4,5.005707,-0.038615,0.032738,-0.012847
...,...,...,...,...
994,996.077421,-6.947498,2.820263,-2.994478
995,997.078503,-6.954477,2.823079,-2.997490
996,998.079586,-6.961456,2.825895,-3.000501
997,999.080669,-6.968434,2.828710,-3.003513


In [389]:
p_x["C"] = "Predict"
p_y["C"] = "True"

d = pd.concat([p_x,p_y],axis=0)

In [390]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C')
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()

In [391]:
def plotting(x, y, z):
    
    fig = plt.figure()
    
    ax = plt.axes(projection="3d")
    plt.axis('equal')
    
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    ax.plot3D(x, y, z, 'red')

    # ax.scatter3D(x, y, z)

    # Showing the above plot
    plt.show()